# module import

In [ ]:
# basic libray
import os
import numpy as np
import random
import pandas as pd
import warnings; warnings.filterwarnings("ignore")

# surprise Library import
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds

# model save
import pickle
import joblib

random.seed(0)

data_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"
model_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/model/"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Read data

In [ ]:
demo = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_01_DEMO.csv") # 고객정보
pdde = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_02_PDDE.csv") # 유통사 상품 구매 내역
product = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_04_PD_CLAC.csv") # 유통사 상품 카테고리 마스터

# preprocessing

In [ ]:
# 상품코드에 따른 상품명 labeling feature

pdde = pdde.merge(product, on = "pd_c", how = "left")
pdde = pdde.drop("pd_c", axis = 1)

In [ ]:
# 결제 내역이 10번 초과인 고객들에 대한 유통사 구매 데이터만 추출

cust_ = pdde.groupby("cust")["rct_no"].nunique()
pdde = pdde.merge(cust_.reset_index().rename(columns = {'rct_no':"pay_cnt"}), on = "cust")
pdde = pdde.query("pay_cnt > 10")
pdde.head(5)

In [ ]:
# surprise 패키지 input을 위해 user-item 행렬 만들기 (surprise 패키지는 행렬의 unstack형식 사용)

user_item_matrix = pd.pivot_table(data = pdde, index = "pd_nm", columns = "cust",
                    values = "buy_ct", aggfunc = "sum")   # 고객의 상품별 구매횟수 -> 상품에 대한 호감도 -> 평점데이터 대체
user_item_matrix = user_item_matrix.apply(lambda x : x/x.sum())  # 고객별 구매횟수에 대한 정규화
user_item_matrix = user_item_matrix.unstack().reset_index().rename(columns = {'cust':"custID", "pd_nm":"item", 0:"buy_ct"})
user_item_matrix = user_item_matrix.dropna()
user_item_matrix.head(5)

# train test dataset split

In [ ]:
# demo 데이터의 고객 중 main.ipynb에서 모델을 적용해볼 고객 5명 추출 후 input dataset에서 제외

val_cust_list = []
for _ in range(10):
    val_cust_list.append(demo.cust.unique()[random.randint(0, demo.shape[0])])
    
input_data = user_item_matrix.query("custID not in @val_cust_list")

In [ ]:
# main.ipynb에서 모델 적용해볼 고객 5명에 대한 data 저장

user_item_matrix.query("custID in @val_cust_list").to_csv(data_path + "custom_dataset/collab_fitering_valset.csv")

In [ ]:
user_item_matrix.query("custID in @val_cust_list").custID.unique()

In [ ]:
# surprise 패키지 input용으로 data 변환
reader = Reader(rating_scale=(0.0, 1.))
data = Dataset.load_from_df(input_data, reader)

# train, test 데이터셋 분할 -> 75:25 비율
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# cross validation

In [ ]:
# surprise 모델 import
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, 
from surprise import KNNBasic,KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
# 교차검증 모듈
from surprise.model_selection import cross_validate


benchmark = []
# 모든 알고리즘을 literate화 시켜서 반복문을 실행시킨다.
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(),
                  KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    
    # 교차검증을 수행하는 단계.
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # 결과 저장과 알고리즘 이름 추가.
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

* 각 모델별 교차검증 점수에 따르면 BaselineOnly 모델의 test_rmse와 fit_timed이 압도적이므로 최종 모델로 선택한다.
* 하지만 주어진 데이터셋을 넘어 대용량 데이터셋 이용을 위해 메모리 절약 효과가 있는 SVD(잠재요인 추출) 모델 또한 선택한다.

# Hyperparameters tunning

In [ ]:
# 이제 trainset과 testset 전체를 사용하여 학습하기 위해 trainset을 전체로 변환
trainset = data.build_full_trainset()

In [ ]:
# SVD 모델에 대해 GridSearch를 사용한 교차검증 rmse를 사용해 하이퍼파라미터 선정
from surprise.model_selection import GridSearchCV

# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) 
gs.fit(data)

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

In [ ]:
# BaselinOnly 모델과 최적 하아퍼파라미터를 적용한 SVD 모델 학습
base = BaselineOnly(); svd = SVD(**gs.best_params['rmse'])
base.fit(trainset); svd.fit(trainset)

# save model

In [ ]:
joblib.dump(base, model_path + 'base.pkl')
joblib.dump(svd, model_path + 'svd.pkl')